In [1]:
import sqlite3
import requests
import pandas as pd

server = 'https://data.public.lu/fr/datasets/r/5e3ee76e-f206-4e50-8c4c-4bcffc32f2ed'


In [2]:
response = requests.get(server)


In [3]:
response.json()

{'age_bathing_waters_0': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Point',
     'coordinates': [6.361894732663164, 49.48994856649354]},
    'properties': {'NAME': 'Lac Remerschen Zone 3',
     'OBJECTID': 10,
     '_subtype': 'age_bathing_waters_0',
     'Hyperlinks': 'http://geoportail.eau.etat.lu/pdf/protection/Badegewaesser/ProfilRemerschen.pdf'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point',
     'coordinates': [6.358443856876467, 49.49095986172581]},
    'properties': {'NAME': 'Lac Remerschen Zone 1',
     'OBJECTID': 12,
     '_subtype': 'age_bathing_waters_0',
     'Hyperlinks': 'http://geoportail.eau.etat.lu/pdf/protection/Badegewaesser/ProfilRemerschen.pdf'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point',
     'coordinates': [6.360573367860487, 49.49195434453994]},
    'properties': {'NAME': 'Lac Remerschen Zone 2',
     'OBJECTID': 13,
     '_subtype': 'age_bathing_waters_0',
     'Hyperlinks': 'http://ge

In [4]:
data = pd\
    .json_normalize(response.json()['age_bathing_waters_0']['features'])\
    .rename(columns={
            "properties.NAME": "name",
            "geometry.coordinates": "coordinates",
            "properties.OBJECTID": "id"
    })\
    .set_index("id")

data['alternate_name'] = data['name']
data[['long', 'lat']] = data['coordinates'].apply(pd.Series)
data = data.reindex(columns=["name", "alternate_name", "long", "lat"])
data

,name,alternate_name,long,lat
id,,,,
10,Lac Remerschen Zone 3,Lac Remerschen Zone 3,6.361895,49.489949
12,Lac Remerschen Zone 1,Lac Remerschen Zone 1,6.358444,49.490960
13,Lac Remerschen Zone 2,Lac Remerschen Zone 2,6.360573,49.491954
11,Lac Weiswampach Zone 2,Lac Weiswampach Zone 2,6.064593,50.138734
14,Lac Weiswampach Zone 1,Lac Weiswampach Zone 1,6.066764,50.137747
15,Liefrange,Liefrange,5.878629,49.904716
16,Lultzhausen,Lultzhausen,5.889693,49.910075
17,Insenborn,Insenborn,5.884626,49.905091
18,Burfelt,Burfelt,5.858626,49.900187


In [5]:
locations = data.sort_values(by="name")
locations

,name,alternate_name,long,lat
id,,,,
18,Burfelt,Burfelt,5.858626,49.900187
20,Fuussefeld,Fuussefeld,5.869396,49.900845
17,Insenborn,Insenborn,5.884626,49.905091
12,Lac Remerschen Zone 1,Lac Remerschen Zone 1,6.358444,49.490960
13,Lac Remerschen Zone 2,Lac Remerschen Zone 2,6.360573,49.491954
10,Lac Remerschen Zone 3,Lac Remerschen Zone 3,6.361895,49.489949
14,Lac Weiswampach Zone 1,Lac Weiswampach Zone 1,6.066764,50.137747
11,Lac Weiswampach Zone 2,Lac Weiswampach Zone 2,6.064593,50.138734
15,Liefrange,Liefrange,5.878629,49.904716


In [6]:
db = sqlite3.connect("../dataset.sqlite3")
locations.to_sql("locations", db, if_exists="append")


11